In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

ast_path = "csv/ast.csv"
cg_path = "csv/cg.csv"
label_path = "csv/labels.csv"
ast = pd.read_csv(ast_path)
cg = pd.read_csv(cg_path)
label = pd.read_csv(label_path)



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cg['contract'] = cg['contract'].apply(lambda x: x.split('/')[-1])

In [3]:
cg['contract']

0       0x0000000000027f6d87be8ade118d9ee56767d993
1       0x0000000000085d4780b73119b644ae5ecd22b376
2       0x00000000000da14c27c155bb7c1ac9bd7519eb3b
3       0x0000000000377d181a0ebd08590c6b399b272000
4       0x0000000000b3f879cb30fe243b4dfee438691c04
                           ...                    
1705    0x0b38bcec2b7404490a16a9fbbd3a87e14dac0367
1706    0x0b39295cbcbc798fd726d5ddf329e53cf4bdee6d
1707    0x0b3a0edd51ee5e7900666bbfdf139ea55ccf0ca0
1708    0x0b3ce7353326ed31b32d32f1b6d84d44bbb7ee0c
1709    0x0b3d7952779bacf37683b5af2b94755b5de9b792
Name: contract, Length: 1710, dtype: object

In [4]:
ast_cg = pd.merge(ast, cg, on='contract', how='inner')
ast_cg_label = pd.merge(ast_cg, label, left_on='contract', right_on='smart_contract_address', how='inner')

ast_cg_label



,SourceUnit,PragmaDirective,ContractDefinition,VariableDeclaration,ElementaryTypeName,EventDefinition,ParameterList,FunctionDefinition,Block,ExpressionStatement,...,56,57,58,59,60,61,62,63,smart_contract_address,vuln
0,1,1.0,3,10,10,1.0,15,6,8,11.0,...,0.000379,-0.005039,-0.000738,0.002955,0.002001,-0.003278,-0.004173,0.001695,0x0000000000027f6d87be8ade118d9ee56767d993,1
1,1,1.0,3,26,26,3.0,31,13,14,11.0,...,0.000784,-0.001640,0.000969,0.000541,-0.000094,-0.002301,-0.000353,0.000812,0x0000000000085d4780b73119b644ae5ecd22b376,0
2,1,1.0,2,11,10,1.0,11,5,3,5.0,...,0.000668,-0.003673,0.002406,-0.003408,0.001187,-0.002278,-0.005548,-0.000387,0x00000000000da14c27c155bb7c1ac9bd7519eb3b,0
3,1,1.0,1,87,93,9.0,54,18,35,69.0,...,0.002259,-0.003641,-0.001298,-0.001466,0.002021,-0.002237,-0.000511,0.001248,0x0000000000377d181a0ebd08590c6b399b272000,0
4,1,1.0,1,77,82,2.0,34,16,34,36.0,...,0.003437,-0.003959,0.001043,-0.000718,0.000673,-0.004874,-0.003678,0.001017,0x0000000000b3f879cb30fe243b4dfee438691c04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,1,4.0,8,827,826,4.0,369,178,277,415.0,...,0.145137,-0.284707,0.132475,-0.092453,0.071549,-0.110078,-0.320830,-0.081096,0x0b2a23a01be33b8c20204de6d5f2e16a1f3b78e5,1
1700,1,1.0,4,33,33,4.0,27,11,7,8.0,...,0.000345,-0.002488,-0.000974,0.000641,0.001696,-0.003759,-0.001016,0.001632,0x0b2b5b527ba2094a2d68bc37568bc0bfde810525,1
1701,1,1.0,1,56,54,4.0,25,10,34,44.0,...,0.000833,-0.004290,-0.001101,0.001654,0.001941,-0.004002,-0.002972,0.000860,0x0b2bb48d7dca92a752737c6f6f73a77c770e7f06,1
1702,1,1.0,4,87,90,6.0,60,27,27,30.0,...,0.001796,-0.004455,-0.001619,-0.001425,0.000626,-0.004940,-0.002563,-0.000398,0x0b2c6a5eb2171d130fb6856a849ce053f276c1bc,1


In [5]:
ast_cg_label = ast_cg_label.fillna(0)

In [6]:
y = ast_cg_label['vuln']
X = ast_cg_label.drop(['vuln', 'smart_contract_address', 'contract'], axis=1)


In [7]:
ast.shape, cg.shape

((1716, 45), (1710, 65))

In [8]:
X.shape

(1704, 108)

In [9]:
ast_cg_label.iloc[-1]

SourceUnit                                                         1
PragmaDirective                                                  1.0
ContractDefinition                                                 7
VariableDeclaration                                               80
ElementaryTypeName                                                83
                                             ...                    
61                                                         -0.002402
62                                                         -0.001025
63                                                           0.00106
smart_contract_address    0x0b2d5155d11f684cc77a6626dc82431df66b7799
vuln                                                               1
Name: 1703, Length: 111, dtype: object

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [11]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)


RandomForestClassifier()

In [12]:
y_pred = rf.predict(X_test)

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.51      0.64        45
           1       0.93      0.99      0.96       296

    accuracy                           0.92       341
   macro avg       0.89      0.75      0.80       341
weighted avg       0.92      0.92      0.92       341



In [14]:
from collections import Counter

In [15]:
Counter(label['vuln'])

Counter({1: 35228, 0: 12327})

In [17]:
from sklearn.svm import SVC

In [18]:
svc = SVC()

In [19]:
svc.fit(X_train, y_train)

SVC()

In [22]:
y_pred = svc.predict(X_test)

In [24]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        45
           1       0.87      1.00      0.93       296

    accuracy                           0.87       341
   macro avg       0.43      0.50      0.46       341
weighted avg       0.75      0.87      0.81       341



/home/enkai/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/enkai/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/enkai/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the deep learning model
def create_deep_model(input_shape, num_classes):
    model = models.Sequential()

    # Input layer
    model.add(layers.Input(shape=input_shape))

    # Hidden layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Example usage
input_shape = (784,)  # Example for a vector with 784 features (e.g., flattened 28x28 image)
num_classes = 10  # Number of output classes (e.g., for MNIST dataset)
model = create_deep_model(input_shape, num_classes)

# Print model summary
model.summary()


ModuleNotFoundError: No module named 'tensorflow'